## Experimental Model

This note includes clustoring model.
What I do is clustoring post_message of all_data to find similar comments.

But, I might not do clustoring as final submission because the result seems meaningless or difficult to evaluate.

If I could do, I also attoch these experiment to final notebook. But, in this mean time, I omit these model from main notebook. Still atouch them when I could not finish new model.

In [1]:
# format function definition copied from main note
from os import path
import pandas as pd
from datetime import datetime
import pytz as tz
import pickle
from os.path import exists
from sklearn.feature_extraction.text import TfidfVectorizer

def format_data(df, page_label):
    """
    function to clean dataframes.
    this function drop rows which contains NA, 
    and drop page_url, source, post_link column
    instead of dropping page_url and source, I add page_label text.
    
    also create datetime column and remove date, time, time_zone
    :param df: 
    :param page_label:  
    :return df: 
    """
    past_shape = df.shape
    df = df.dropna(axis=0, how='any')
    df = df.drop(['page_url', 'source', 'post_link', 'post_type'], axis=1)
    df['page_label'] = page_label
    df = df.reset_index()  # reset index for datetime append operation

    datetime_list = list()
    for i in range(df.shape[0]):
        # need correct numerical index for code below.
        dt_str = '%s %s %s' % (df['date'][i], df['time'][i], df['time_zone'][i])
        dt_str = dt_str.strip()  # need strip to remove last blank character.
        try:
            dt_parsed = datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S %Z%z')
            CT = tz.timezone('America/Chicago')
            dt_parsed = dt_parsed.astimezone(CT)
        except ValueError as e:
            print(e)
            print('error happen :%s;' % dt_str)
            dt_parsed = None
        datetime_list.append(dt_parsed)

    df['datetime'] = datetime_list
    df = df.drop(['index', 'date', 'time', 'time_zone'], axis=1)
    df['datetime'] = pd.to_datetime(df['datetime'])

    show = True
    if show:
        print('result shape compare: ', past_shape, df.shape)
    return df

# make term document matrix


data_path = '/Users/reiven/Documents/Python/CSCI4710/Project/Data'
binary_path = '/Users/reiven/Documents/Python/CSCI4710/Project/binary_file'
tfidf_vectorizer_path = "/Users/reiven/Documents/Python/CSCI4710/Project/binary_file/tfidf_vectorizer.pkl"

blacktivist_data = pd.read_csv(path.join(data_path, 'blacktivist.csv'))
being_patriotic_data = pd.read_csv(path.join(data_path, 'being_patriotic.csv'))
# heart_of_texas_data = pd.read_csv(path.join(data_path, 'heart_of_texas.csv'))
lgbt_united_data = pd.read_csv(path.join(data_path, 'lgbt_united.csv'))
secured_borders_data = pd.read_csv(path.join(data_path, 'secured_borders.csv'))
united_muslims_of_america_data = pd.read_csv(path.join(data_path, 'united_muslims_of_america.csv'))

# format all data with format_data()
blacktivist = format_data(blacktivist_data, 'blacktivist')
being_patriotic = format_data(being_patriotic_data, 'being_patriotic')
lgbt_united = format_data(lgbt_united_data, 'lgbt_united')
secured_borders = format_data(secured_borders_data, 'secured_borders')
united_muslims_of_america = format_data(united_muslims_of_america_data, 'united_muslims')

all_data = pd.concat([blacktivist, being_patriotic, lgbt_united, secured_borders, united_muslims_of_america])

if exists(tfidf_vectorizer_path):
    with open(tfidf_vectorizer_path, "rb") as f:
        vectorizer = pickle.load(f)
        f.close()
else:
    vectorizer = TfidfVectorizer(
        use_idf=True,
        analyzer='word',
        stop_words='english',
        lowercase=True
    )
    vectorizer.fit(all_data['post_message'])
    with open(tfidf_vectorizer_path, "wb") as f:
        pickle.dump(vectorizer, f)
        f.close()

post_message_tfidf_matrix = vectorizer.transform(all_data['post_message'])


result shape compare:  (500, 10) (498, 5)
result shape compare:  (500, 10) (496, 5)
result shape compare:  (500, 10) (500, 5)
result shape compare:  (500, 10) (500, 5)
result shape compare:  (500, 10) (500, 5)


In [9]:
# KMeans
from sklearn.cluster import MiniBatchKMeans


# choice 5 because the data are from 5 resources
model = MiniBatchKMeans(n_clusters=5)
model.fit(post_message_tfidf_matrix)
predict_label = model.predict(post_message_tfidf_matrix)
print(predict_label[0:10])



[4 1 1 1 1 4 1 1 4 4]


In [11]:
from sklearn.cluster import DBSCAN

# choice 5 because the data are from 5 resources
model = DBSCAN()
predict_label = model.fit_predict(post_message_tfidf_matrix)


print(predict_label[0:10])

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [12]:
from sklearn.mixture import GaussianMixture

matrix_array_path = "/Users/reiven/Documents/Python/CSCI4710/Project/binary_file/matrix_array.pkl"
GMM_path = "/Users/reiven/Documents/Python/CSCI4710/Project/binary_file/gmm.pkl"

# if exists(matrix_array_path):
#     with open(matrix_array_path, "rb") as f:
#         matrix_array = pickle.load(f)
# else: 
#     matrix_array = matrix.toarray()
#     with open(matrix_array_path, "wb") as f:
#         pickle.dump(matrix_array, f)
  
matrix_array = post_message_tfidf_matrix.toarray()  
if exists(GMM_path):
    with open(GMM_path, "rb") as f:
        model = pickle.load(f)
else: 
    model = GaussianMixture(n_components=5, covariance_type='spherical')
    model.fit(matrix_array)
    with open(GMM_path, "wb") as f:
        pickle.dump(model, f)
        f.close()


predict_label = model.predict(matrix_array)

print(predict_label[0:10])

[0 0 0 0 0 0 0 0 0 0]
